In [ ]:
'''
【课程5.5】  折线图与面积图

① 单线图、多线图
② 面积图、堆叠面积图

'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块

from bokeh.plotting import figure,show
# 导入图表绘制、图标展示模块

Loading BokehJS ...

In [2]:
# 1、折线图 - 单线图

from bokeh.models import ColumnDataSource
# 导入ColumnDataSource模块
# 将数据存储为ColumnDataSource对象
# 参考文档：http://bokeh.pydata.org/en/latest/docs/user_guide/data.html
# 可以将dict、Dataframe、group对象转化为ColumnDataSource对象
# 参考链接：https://blog.51cto.com/u_15064628/2600683

df = pd.DataFrame({'value':np.random.randn(100).cumsum()})
# 创建数据


# dic = {'index':df.index.tolist(),'value':df['value'].tolist()}
#source = ColumnDataSource(data = dic)


df.index.name = 'index'
source = ColumnDataSource(data = df)
# 转化为ColumnDataSource对象
# 这里注意了，index和columns都必须有名称字段

p = figure(plot_width=600, plot_height=400)
p.line(x='index',y='value',source = source,     # 设置x，y值, source → 数据源
       line_width=1, line_alpha = 0.8, line_color = 'black',line_dash = [10,4])   # 线型基本设置
# 绘制折线图
p.circle(x='index',y='value',source = source, 
         size = 2,color = 'red',alpha = 0.8)
# 绘制折点

show(p)

In [3]:
# 1、折线图 - 多线图
# ① multi_line

df = pd.DataFrame({'A':np.random.randn(100).cumsum(),"B":np.random.randn(100).cumsum()})
# 创建数据

p = figure(plot_width=600, plot_height=400)
p.multi_line([df.index, df.index], [df['A'], df['B']],   # 注意x，y值的设置 → [x1,x2,x3,..], [y1,y2,y3,...]
             color=["firebrick", "navy"],    # 可同时设置 → color= "firebrick"
             alpha=[0.8, 0.6],     # 可同时设置 → alpha = 0.6
             line_width=[2,1],     # 可同时设置 → line_width = 2
            )
# 绘制多段线
# 这里由于需要输入具体值，故直接用dataframe，或者dict即可

show(p)

In [4]:
# 1、折线图 - 多线图
# ② 多个line

x = np.linspace(0.1, 5, 100)
# 创建x值

p = figure(title="log axis example", y_axis_type="log",y_range=(0.001, 10**22))
# 这里设置对数坐标轴

p.line(x, np.sqrt(x), legend="y=sqrt(x)",
       line_color="tomato", line_dash="dotdash")
# line1

p.line(x, x, legend="y=x")
p.circle(x, x, legend="y=x")
# line2，折线图+散点图

p.line(x, x**2, legend="y=x**2")
p.circle(x, x**2, legend="y=x**2",fill_color=None, line_color="olivedrab")
# line3

p.line(x, 10**x, legend="y=10^x",line_color="gold", line_width=2)
# line4

p.line(x, x**x, legend="y=x^x",line_dash="dotted", line_color="indigo", line_width=2)
# line5

p.line(x, 10**(x**2), legend="y=10^(x^2)",line_color="coral", line_dash="dashed", line_width=2)
# line6

p.legend.location = "top_left"
p.xaxis.axis_label = 'Domain'
p.yaxis.axis_label = 'Values (log scale)'
# 设置图例及label

show(p)

In [5]:
# 2、面积图 - 单维度面积图

s = pd.Series(np.random.randn(100).cumsum())
s.iloc[0] = 0     #第一个值
s.iloc[-1] = 0    #最后一个值
# 创建数据
# 注意设定起始值和终点值为最低点

p = figure(plot_width=600, plot_height=400)
p.patch(s.index, s.values,     # 设置x，y值
        line_width=1, line_alpha = 0.8, line_color = 'green',line_dash = [10,4],   # 线型基本设置
        fill_color = 'navy',fill_alpha = 0.2
        )
# 绘制面积图
# .patch将会把所有点连接成一个闭合面

p.circle(s.index, s.values,size = 5,color = 'red',alpha = 0.8)
# 绘制折点

show(p)

In [33]:
# 2、面积图 - 面积堆叠图,坐标实现的顺序为19-0,0-19

from bokeh.palettes import brewer
# 导入brewer模块，即颜色盘

N = 20
cats = 10
rng = np.random.RandomState(1)
df = pd.DataFrame(rng.randint(10, 100, size=(N, cats))).add_prefix('y')
# 创建数据，shape为（20，10）,因为没有对列名和行名进行指定，所以默认为0-9数字
# df

df_top = df.cumsum(axis=1)   # 每一个堆叠面积图的最高点 
df_bottom = df_top.shift(axis=1).fillna({'y0': 0})[::-1]  # 每一个堆叠面积图的最低点，并反向
df_stack = pd.concat([df_bottom, df_top], ignore_index=True)   # 数据合并，每一组数据都是一个可以围合成一个面的散点集合
# 得到堆叠面积数据

colors = brewer['Spectral'][df_stack.shape[1]]    # 根据变量数拆分颜色
x = np.hstack((df.index[::-1], df.index))         # 得到围合顺序的index，这里由于一列是20个元素，所以连接成面需要40个点

p = figure(x_range=(0, N-1), y_range=(0, 700))
p.patches([x] * df_stack.shape[1],                       # 得到10组index
          [df_stack[c].values for c in df_stack],     # c为df_stack的列名，这里得到10组对应的valyes
          color=colors, alpha=0.8, line_color=None)   # 设置其他参数

show(p)


In [31]:
df_stack

,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9
0,0.0,55.0,132.0,208.0,296.0,352.0,362.0,401.0,474.0,559.0
1,0.0,87.0,146.0,203.0,290.0,340.0,428.0,483.0,580.0,606.0
2,0.0,25.0,59.0,78.0,154.0,235.0,298.0,377.0,423.0,454.0
3,0.0,93.0,128.0,152.0,175.0,269.0,322.0,338.0,425.0,491.0
4,0.0,32.0,55.0,118.0,212.0,232.0,297.0,368.0,434.0,533.0
5,0.0,76.0,157.0,215.0,279.0,304.0,319.0,346.0,398.0,428.0
6,0.0,75.0,145.0,179.0,271.0,283.0,303.0,367.0,459.0,555.0
7,0.0,23.0,43.0,76.0,167.0,184.0,218.0,302.0,332.0,374.0
8,0.0,86.0,139.0,169.0,209.0,255.0,272.0,327.0,405.0,472.0
9,0.0,95.0,175.0,187.0,273.0,304.0,389.0,406.0,493.0,575.0


In [32]:
[c for c in df_stack]

['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9']

In [26]:
# 2、面积图 - 面积堆叠图,坐标的组成顺序为0-19,19-0
'''
即一个面积图需要上下两个包络面组成，所以一个面积图的形成同时需要一个坐标的最小值和最大值
'''

from bokeh.palettes import brewer
# 导入brewer模块，即颜色盘

N = 20
cats = 10
rng = np.random.RandomState(1)
df = pd.DataFrame(rng.randint(10, 100, size=(N, cats))).add_prefix('y')
# 创建数据，shape为（20，10）,因为没有对列名和行名进行指定，所以默认为0-9数字
# df

df_top = df.cumsum(axis=1)   # 每一个堆叠面积图的最高点
df_top_x = df.cumsum(axis=1)[::-1]   
df_bottom = df_top.shift(axis=1).fillna({'y0': 0})  # 每一个堆叠面积图的最低点，并反向
df_stack = pd.concat([df_bottom, df_top_x], ignore_index=True)   # 数据合并，每一组数据都是一个可以围合成一个面的散点集合
# 得到堆叠面积数据

colors = brewer['Spectral'][df_stack.shape[1]]    # 根据变量数拆分颜色
x = np.hstack((df.index, df.index[::-1]))         # 得到围合顺序的index，这里由于一列是20个元素，所以连接成面需要40个点

p = figure(x_range=(0, N-1), y_range=(0, 700))
p.patches([x] * df_stack.shape[1],                       # 得到10组index
          [df_stack[c].values for c in df_stack],     # c为df_stack的列名，这里得到10组对应的valyes
          color=colors, alpha=0.8, line_color=None)   # 设置其他参数

show(p)